# GAN Implementation
Author: Taylor Qin<br>
Version: v1-14APR

In [ ]:
# example of training a gan on mnist
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
 
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
 
# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    # upsample to 14x14
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 28x28
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
    return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model
 
# load and prepare mnist training images
def load_real_samples():
    # load mnist dataset
    (trainX, _), (_, _) = load_data()
    # expand to 3d, e.g. add channels dimension
    X = expand_dims(trainX, axis=-1)
    # convert from unsigned ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [0,1]
    X = X / 255.0
    return X
 
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y
 
# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()
 
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)
 
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # create training set for the discriminator
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            # update discriminator model weights
            d_loss, _ = d_model.train_on_batch(X, y)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
        # evaluate the model performance, sometimes
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)
 
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 1/234, d=0.711, g=0.670
>1, 2/234, d=0.701, g=0.691
>1, 3/234, d=0.688, g=0.705
>1, 4/234, d=0.679, g=0.726
>1, 5/234, d=0.677, g=0.741
>1, 6/234, d=0.660, g=0.760
>1, 7/234, d=0.654, g=0.777
>1, 8/234, d=0.642, g=0.795
>1, 9/234, d=0.640, g=0.806
>1, 10/234, d=0.637, g=0.822
>1, 11/234, d=0.629, g=0.834
>1, 12/234, d=0.625, g=0.838
>1, 13/234, d=0.626, g=0.833
>1, 14/234, d=0.629, g=0.817
>1, 15/234, d=0.628, g=0.798
>1, 16/234, d=0.632, g=0.775
>1, 17/234, d=0.628, g=0.756
>1, 18/234, d=0.632, g=0.743
>1, 19/234, d=0.633, g=0.729
>1, 20/234, d=0.628, g=0.718
>1, 21/234, d=0.620, g=0.710
>1, 22/234, d=0.617, g=0.704
>1, 23/234, d=0.613, g=0.701
>1, 24/234, d=0.605, g=0.698
>1, 25/234, d=0.600, g=0.696
>1, 26/234, d=0.590, g=0.697
>1, 27/234, d=0.580, g=0.697
>1, 28/234, d=0.576, g=0.697
>1, 29/234, d=0.562, g=0.697
>1, 30/234, d=0.552, g=0.698
>1, 31/234, d=0.540, g=0.699
>1, 32/234, d=0.528, g=0.700
>1, 33/234, d=0.525, g=0.701
>1, 34/234, d=0.516, g=0.702
>1, 35/234, d=0.505, g=

In [ ]:
# import numpy as np
# from keras.datasets import mnist
# from keras.models import Model
# from keras.layers import Dense, Input
# import matplotlib.pyplot as plt

# # define the standalone discriminator model
# def define_discriminator(in_shape=(28,28,1)):
#     model = Sequential()
#     model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dropout(0.4))
#     model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Dropout(0.4))
#     model.add(Flatten())
#     model.add(Dense(1, activation='sigmoid'))
#     # compile model
#     opt = Adam(lr=0.0002, beta_1=0.5)
#     model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
#     return model

# from keras.models import Sequential
# from keras.optimizers import Adam
# from keras.layers import Dense
# from keras.layers import Conv2D
# from keras.layers import Flatten
# from keras.layers import Dropout
# from keras.layers import LeakyReLU
# from keras.utils.vis_utils import plot_model

# def load_real_samples():
#     (trainX, _), (_, _) = mnist.load_data()
#     # expand to 3d, e.g. add channels dimension
#     X = expand_dims(trainX, axis=-1)
#     # convert from unsigned ints to floats
#     X = X.astype('float32')
#     # scale from [0,255] to [0,1]
#     X = X / 255.0
#     return X

# def generate_real_samples(dataset, n_samples):
#     # choose random instances
#     ix = randint(0, dataset.shape[0], n_samples)
#     # retrieve selected images
#     X = dataset[ix]
#     # generate 'real' class labels (1)
#     y = ones((n_samples, 1))
#     return X, y


# # generate n fake samples with class labels
# def generate_fake_samples(n_samples):
#     # generate uniform random numbers in [0,1]
#     X = rand(28 * 28 * n_samples)
#     # reshape into a batch of grayscale images
#     X = X.reshape((n_samples, 28, 28, 1))
#     # generate 'fake' class labels (0)
#     y = zeros((n_samples, 1))
#     return X, y

# # train the discriminator model
# def train_discriminator(model, dataset, n_iter=100, n_batch=256):
#     half_batch = int(n_batch / 2)
#     # manually enumerate epochs
#     for i in range(n_iter):
#         # get randomly selected 'real' samples
#         X_real, y_real = generate_real_samples(dataset, half_batch)
#         # update discriminator on real samples
#         _, real_acc = model.train_on_batch(X_real, y_real)
#         # generate 'fake' examples
#         X_fake, y_fake = generate_fake_samples(half_batch)
#         # update discriminator on fake samples
#         _, fake_acc = model.train_on_batch(X_fake, y_fake)
#         # summarize performance
#         print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))

# def define_generator(latent_dim):
#     model = Sequential()
#     # foundation for 7x7 image
#     n_nodes = 128 * 7 * 7
#     model.add(Dense(n_nodes, input_dim=latent_dim))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Reshape((7, 7, 128)))
#     # upsample to 14x14
#     model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     # upsample to 28x28
#     model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
#     model.add(LeakyReLU(alpha=0.2))
#     model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
#     return model

# # generate points in latent space as input for the generator
# def generate_latent_points(latent_dim, n_samples):
#     # generate points in the latent space
#     x_input = randn(latent_dim * n_samples)
#     # reshape into a batch of inputs for the network
#     x_input = x_input.reshape(n_samples, latent_dim)
#     return x_input

# def generate_fake_samples(g_model, latent_dim, n_samples):
#     # generate points in latent space
#     x_input = generate_latent_points(latent_dim, n_samples)
#     # predict outputs
#     X = g_model.predict(x_input)
#     # create 'fake' class labels (0)
#     y = zeros((n_samples, 1))
#     return X, y

# # define the combined generator and discriminator model, for updating the generator
# def define_gan(g_model, d_model):
#     # make weights in the discriminator not trainable
#     d_model.trainable = False
#     # connect them
#     model = Sequential()
#     # add generator
#     model.add(g_model)
#     # add the discriminator
#     model.add(d_model)
#     # compile model
#     opt = Adam(lr=0.0002, beta_1=0.5)
#     model.compile(loss='binary_crossentropy', optimizer=opt)
#     return model

# # train the composite model
# def train_gan(gan_model, latent_dim, n_epochs=100, n_batch=256):
#     # manually enumerate epochs
#     for i in range(n_epochs):
#         # prepare points in latent space as input for the generator
#         x_gan = generate_latent_points(latent_dim, n_batch)
#         # create inverted labels for the fake samples
#         y_gan = ones((n_batch, 1))
#         # update the generator via the discriminator's error
#         gan_model.train_on_batch(x_gan, y_gan)

# def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
#     bat_per_epo = int(dataset.shape[0] / n_batch)
#     half_batch = int(n_batch / 2)
#     # manually enumerate epochs
#     for i in range(n_epochs):
#         # enumerate batches over the training set
#         for j in range(bat_per_epo):
#             # get randomly selected 'real' samples
#             X_real, y_real = generate_real_samples(dataset, half_batch)
#             # generate 'fake' examples
#             X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
#             # create training set for the discriminator
#             X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
#             # update discriminator model weights
#             d_loss, _ = d_model.train_on_batch(X, y)
#             # prepare points in latent space as input for the generator
#             X_gan = generate_latent_points(latent_dim, n_batch)
#             # create inverted labels for the fake samples
#             y_gan = ones((n_batch, 1))
#             # update the generator via the discriminator's error
#             g_loss = gan_model.train_on_batch(X_gan, y_gan)
#             # summarize loss on this batch
#             print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))

# # evaluate the discriminator, plot generated images, save generator model
# def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
#     # prepare real samples
#     X_real, y_real = generate_real_samples(dataset, n_samples)
#     # evaluate discriminator on real examples
#     _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
#     # prepare fake examples
#     x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
#     # evaluate discriminator on fake examples
#     _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
#     # summarize discriminator performance
#     print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
#     # save plot
#     save_plot(x_fake, epoch)
#     # save the generator model tile file
#     filename = 'generator_model_%03d.h5' % (epoch + 1)
#     g_model.save(filename)
    
# if __name__ == '__main__':
# #     (x_train, y_train), (x_test, y_test) = mnist.load_data()
# #     model = define_discriminator()
#     model = define_discriminator()
# # load image data
#     dataset = load_real_samples()
# # fit the model
#     train_discriminator(model, dataset)
#     latent_dim = 100
    
# # define the discriminator model
#     model = define_generator(latent_dim)
# # generate samples
#     n_samples = 25
#     X, _ = generate_fake_samples(model, latent_dim, n_samples)
# # plot the generated samples
#     for i in range(n_samples):
#         # define subplot
#         plt.subplot(5, 5, 1 + i)
#         # turn off axis labels
#         plt.axis('off')
# 	# plot single image
#         plt.imshow(X[i, :, :, 0], cmap='gray_r')
# # show the figure
# plt.show()
# #     # Prepocess
# #     x_train = x_train.astype('float32') / 255.
# #     x_test = x_test.astype('float32') / 255.
# #     x_train = x_train.reshape((x_train.shape[0], -1))
# #     x_test = x_test.reshape((x_test.shape[0], -1))

# #     # Train
# #     encoder, autoencoder = train(x_train=x_train)
# #     encode_images = encoder.predict(x_test)
# #     decode_images = autoencoder.predict(x_test)
# #     plot_images(decode_images, x_test)